In [1]:
import requests
import pandas as pd
import time
import json
from pathlib import Path
from typing import Dict, Any, List
from datetime import datetime
import socket
from urllib.parse import urlparse

In [ ]:
API_BASE_URL = "http://localhost:8000"
AIRFLOW_URL = "http://localhost:8080"

# Host & port các service
POSTGRES_HOST = "localhost"
POSTGRES_PORT = 5432          # hoặc 5433 nếu muốn connect airflow-db
REDIS_HOST = "localhost"
REDIS_PORT = 6379
OPENSEARCH_HOST = "http://localhost:9200"
OLLAMA_HOST = "http://localhost:11434"

# # Đường dẫn files
# DATA_DIR = Path("data")
# PDF_DIR = DATA_DIR / "pdf"
# QUESTION_FILE = DATA_DIR / "q.csv"
# OUTPUT_FILE = DATA_DIR / "answers_output.csv"

# Cấu hình RAG
TOP_K = 5
USE_HYBRID = True
TIMEOUT = 60

In [3]:
def print_header(text: str):
    print("\n" + "=" * 80)
    print(f"  {text}")
    print("=" * 80)

def print_step(step_num: int, text: str):
    print(f"\n{'─' * 80}")
    print(f"📌 BƯỚC {step_num}: {text}")
    print(f"{'─' * 80}")

def check_system_health() -> bool:
    """Kiểm tra health của hệ thống."""
    print_step(0, "Kiểm tra hệ thống")

    services = {
        "api": f"{API_BASE_URL}/health",
        "postgres": f"postgresql://{POSTGRES_HOST}:{POSTGRES_PORT}",
        "redis": f"redis://{REDIS_HOST}:{REDIS_PORT}",
        "opensearch": f"{OPENSEARCH_HOST}",
        "ollama": f"{OLLAMA_HOST}",
    }

    all_healthy = True
    for service, url in services.items():
        try:
            if service == "api":
                response = requests.get(url, timeout=5)
                status = response.json().get("status", "unknown")
            else:
                # Chỉ kiểm tra port mở
                parsed = urlparse(url)
                host = parsed.hostname
                port = parsed.port
                sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                sock.settimeout(2)
                sock.connect((host, port))
                sock.close()
                status = "healthy"

            icon = "✅" if status in ["healthy", "ready", "connected"] else "❌"
            print(f"  {icon} {service}: {status}")
            if status not in ["healthy", "ready", "connected"]:
                all_healthy = False
        except Exception as e:
            print(f"  ❌ {service}: {e}")
            all_healthy = False

    if not all_healthy:
        print("\n⚠️  CẢNH BÁO: Một số services chưa sẵn sàng!")
        print("Vui lòng chạy: docker compose up -d")
        return False

    print("\n✅ Hệ thống hoạt động bình thường!")
    return True

In [4]:
def check_pdf_files() -> List[Path]:
    """Kiểm tra PDF files trong folder."""
    print_step(1, "Kiểm tra PDF files")

    if not PDF_DIR.exists():
        print(f"\n⚠️  Thư mục không tồn tại: {PDF_DIR}")
        PDF_DIR.mkdir(parents=True, exist_ok=True)
        print(f"✅ Đã tạo thư mục: {PDF_DIR}")

    pdf_files = list(PDF_DIR.glob("*.pdf"))

    if pdf_files:
        print(f"\n✅ Tìm thấy {len(pdf_files)} file(s) PDF:")
        for i, pdf_file in enumerate(pdf_files, 1):
            size_mb = pdf_file.stat().st_size / (1024 * 1024)
            print(f"  {i}. {pdf_file.name} ({size_mb:.2f} MB)")
    else:
        print(f"\n⚠️  Chưa có PDF nào trong: {PDF_DIR}")
        print("Vui lòng copy PDF files vào folder này!")
        return []

    return pdf_files

In [5]:
def trigger_pdf_processing():
    """Hướng dẫn trigger Airflow DAG để xử lý PDF."""
    print_step(2, "Xử lý PDF files")

    print("\n📝 Để xử lý PDF, vui lòng chọn một trong hai cách:")
    print("\n1️⃣  Sử dụng Airflow UI:")
    print(f"   - Mở: {AIRFLOW_URL}")
    print("   - Login: admin / admin")
    print("   - Tìm DAG: 'pdf_ingestion_dag'")
    print("   - Click nút 'Trigger DAG' (▶️)")

    print("\n2️⃣  Sử dụng CLI:")
    print("   docker exec rag-mcq-airflow airflow dags trigger pdf_ingestion_dag")

    print("\n⏱️  Thời gian xử lý: ~2-5 phút/PDF")
    print("\n📊 Quy trình:")
    print("   → Parse PDF (Docling + GROBID)")
    print("   → Chunk documents (section-aware)")
    print("   → Generate embeddings (768D)")
    print("   → Index to OpenSearch")

    # Hỏi user có muốn đợi không
    input("\n⏸️  Nhấn ENTER sau khi đã trigger DAG và đợi xử lý xong...")

In [6]:
def check_indexed_documents() -> int:
    """Kiểm tra số lượng documents đã được index."""
    print_step(3, "Kiểm tra documents đã index")

    try:
        opensearch_auth = ("admin", "admin")
        response = requests.get(
            "http://localhost:9200/mcq-documents/_count",
            auth=opensearch_auth,
            verify=False,
            timeout=10,
        )

        if response.status_code == 200:
            count = response.json().get("count", 0)
            print(f"\n📄 Số lượng document chunks: {count}")

            if count > 0:
                print("✅ Hệ thống đã sẵn sàng để trả lời câu hỏi!")
            else:
                print("⚠️  Chưa có documents nào. Vui lòng chạy Airflow DAG trước!")

            return count
        else:
            print(f"❌ Lỗi khi query OpenSearch: {response.status_code}")
            return 0

    except Exception as e:
        print(f"❌ Không thể kết nối OpenSearch: {e}")
        return 0

In [7]:
def load_questions() -> pd.DataFrame:
    """Đọc câu hỏi từ CSV file."""
    print_step(4, "Đọc câu hỏi từ CSV")

    if not QUESTION_FILE.exists():
        print(f"\n❌ File không tồn tại: {QUESTION_FILE}")
        print("Vui lòng tạo file question.csv với format:")
        print("  Question,A,B,C,D,source_folder")
        return None

    try:
        df = pd.read_csv(QUESTION_FILE)
        print(f"\n✅ Đọc thành công {len(df)} câu hỏi")

        # Kiểm tra columns
        required_cols = ["Question", "A", "B", "C", "D"]
        missing_cols = [col for col in required_cols if col not in df.columns]

        if missing_cols:
            print(f"❌ Thiếu columns: {missing_cols}")
            return None

        # Hiển thị sample
        print("\n📝 Ví dụ câu hỏi đầu tiên:")
        print(f"   Q: {df.iloc[0]['Question'][:100]}...")

        return df

    except Exception as e:
        print(f"❌ Lỗi khi đọc CSV: {e}")
        return None

In [8]:
def ask_single_question(
    question: str, options: Dict[str, str], source_folder: str = None
) -> Dict[str, Any]:
    """Trả lời một câu hỏi MCQ."""

    payload = {"question": question, "options": options, "top_k": TOP_K, "use_hybrid": USE_HYBRID}

    if source_folder and pd.notna(source_folder):
        payload["source_folder"] = source_folder

    try:
        response = requests.post(f"{API_BASE_URL}/api/v1/ask", json=payload, timeout=TIMEOUT)
        response.raise_for_status()
        return response.json()

    except requests.exceptions.Timeout:
        print(f"    ⏱️  Timeout (>{TIMEOUT}s)")
        return {"error": "timeout", "predicted_option": None}

    except Exception as e:
        print(f"    ❌ Lỗi: {str(e)[:100]}")
        return {"error": str(e), "predicted_option": None}

In [9]:
def answer_all_questions(df: pd.DataFrame) -> pd.DataFrame:
    """Trả lời tất cả câu hỏi trong dataframe."""
    print_step(5, "Trả lời câu hỏi")

    results = []
    total = len(df)

    print(f"\n🎯 Bắt đầu trả lời {total} câu hỏi...\n")

    start_time = time.time()

    for idx, row in df.iterrows():
        q_num = idx + 1

        # Progress
        progress = (q_num / total) * 100
        print(f"[{q_num}/{total}] ({progress:.1f}%) ", end="")
        print(f"Q: {row['Question'][:60]}...")

        # Chuẩn bị options
        options = {"A": str(row["A"]), "B": str(row["B"]), "C": str(row["C"]), "D": str(row["D"])}

        # Lấy source_folder nếu có
        source_folder = row.get("source_folder", None)

        # Gọi API
        q_start = time.time()
        answer_data = ask_single_question(
            question=row["Question"], options=options, source_folder=source_folder
        )
        q_time = time.time() - q_start

        # Parse kết quả
        predicted = answer_data.get("predicted_option", "N/A")
        confidence = answer_data.get("confidence", "unknown")
        reasoning = answer_data.get("reasoning", "")
        error = answer_data.get("error", None)

        # Hiển thị kết quả
        conf_icon = {"high": "🟢", "medium": "🟡", "low": "🔴"}.get(confidence, "⚪")

        print(f"    → Đáp án: {predicted} {conf_icon} ({q_time:.1f}s)")

        # Lưu kết quả
        result = {
            "question_number": q_num,
            "question": row["Question"],
            "option_A": row["A"],
            "option_B": row["B"],
            "option_C": row["C"],
            "option_D": row["D"],
            "source_folder": source_folder if pd.notna(source_folder) else "",
            "predicted_answer": predicted,
            "confidence": confidence,
            "reasoning": reasoning[:500] if reasoning else "",  # Giới hạn độ dài
            "processing_time_seconds": round(q_time, 2),
            "error": error if error else "",
            "timestamp": datetime.now().isoformat(),
        }

        # Thêm timing details nếu có
        if "timing" in answer_data:
            timing = answer_data["timing"]
            result["retrieval_time_ms"] = timing.get("retrieval_ms", 0)
            result["generation_time_ms"] = timing.get("generation_ms", 0)

        results.append(result)

        # Ngắt dòng sau mỗi câu
        if q_num % 5 == 0:
            print()

    total_time = time.time() - start_time

    print("\n" + "─" * 80)
    print(f"✅ Hoàn thành! Tổng thời gian: {total_time:.1f}s")
    print(f"⏱️  Trung bình: {total_time / total:.1f}s/câu")
    print("─" * 80)

    return pd.DataFrame(results)

In [10]:
def save_results(results_df: pd.DataFrame):
    """Lưu kết quả ra CSV file."""
    print_step(6, "Lưu kết quả")

    try:
        results_df.to_csv(OUTPUT_FILE, index=False, encoding="utf-8-sig")

        print(f"\n✅ Đã lưu kết quả vào: {OUTPUT_FILE}")
        print(f"📊 Tổng số câu: {len(results_df)}")

        # Thống kê
        conf_counts = results_df["confidence"].value_counts()
        print("\n📈 Thống kê độ tin cậy:")
        for conf, count in conf_counts.items():
            percentage = (count / len(results_df)) * 100
            print(f"  • {conf}: {count} câu ({percentage:.1f}%)")

        # Kiểm tra errors
        errors = results_df[results_df["error"] != ""]
        if len(errors) > 0:
            print(f"\n⚠️  Có {len(errors)} câu bị lỗi:")
            for idx, row in errors.iterrows():
                print(f"  • Câu {row['question_number']}: {row['error'][:50]}")

    except Exception as e:
        print(f"❌ Lỗi khi lưu file: {e}")

In [11]:
def display_summary(results_df: pd.DataFrame):
    """Hiển thị tóm tắt kết quả."""
    print_header("📊 TÓM TẮT KẾT QUẢ")

    total = len(results_df)
    successful = len(results_df[results_df["error"] == ""])

    print(f"\n✅ Tổng số câu: {total}")
    print(f"✅ Trả lời thành công: {successful}")
    print(f"❌ Lỗi: {total - successful}")

    # Top 5 câu có confidence cao
    high_conf = results_df[results_df["confidence"] == "high"].head(5)
    if len(high_conf) > 0:
        print(f"\n🟢 Top {len(high_conf)} câu có độ tin cậy cao:")
        for idx, row in high_conf.iterrows():
            print(
                f"  {row['question_number']}. Đáp án {row['predicted_answer']}: {row['question'][:60]}..."
            )

    # Câu có confidence thấp
    low_conf = results_df[results_df["confidence"] == "low"]
    if len(low_conf) > 0:
        print(f"\n🔴 {len(low_conf)} câu có độ tin cậy thấp:")
        for idx, row in low_conf.head(3).iterrows():
            print(f"  {row['question_number']}. {row['question'][:60]}...")

    print("\n" + "=" * 80)

In [12]:
def main():
    """Hàm chính để chạy toàn bộ pipeline."""

    print_header("🚀 RAG MCQ SYSTEM - AUTOMATIC ANSWERING")
    print(f"\n📅 Bắt đầu: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

    # Step 0: Check system health
    if not check_system_health():
        print("\n❌ Hệ thống chưa sẵn sàng. Vui lòng khởi động services trước!")
        return

    # Step 1: Check PDF files
    pdf_files = check_pdf_files()
    if not pdf_files:
        print("\n⚠️  Không có PDF nào để xử lý!")
        proceed = input("Bạn có muốn tiếp tục trả lời câu hỏi không? (y/n): ")
        if proceed.lower() != "y":
            return

    # Step 2: Process PDFs (manual trigger)
    if pdf_files:
        trigger_pdf_processing()

    # Step 3: Check indexed documents
    doc_count = check_indexed_documents()
    if doc_count == 0:
        print("\n❌ Không có documents nào trong hệ thống!")
        print("Không thể trả lời câu hỏi. Vui lòng xử lý PDF trước!")
        return

    # Step 4: Load questions
    questions_df = load_questions()
    if questions_df is None:
        return

    # Step 5: Answer all questions
    results_df = answer_all_questions(questions_df)

    # Step 6: Save results
    save_results(results_df)

    # Display summary
    display_summary(results_df)

    print_header("✅ HOÀN TẤT!")
    print(f"\n📁 Kết quả đã được lưu tại: {OUTPUT_FILE.absolute()}")
    print(f"📅 Kết thúc: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print("\n" + "=" * 80 + "\n")

In [13]:
# if __name__ == "__main__":
#     try:
#         main()
#     except KeyboardInterrupt:
#         print("\n\n⚠️  Đã dừng bởi người dùng (Ctrl+C)")
#     except Exception as e:
#         print(f"\n\n❌ Lỗi không mong đợi: {e}")
#         import traceback

#         traceback.print_exc()

In [14]:
from pathlib import Path
from src.config import get_settings

settings = get_settings()
pdf_dir = Path(settings.data.pdf_dir)

pdf_files = list(pdf_dir.glob("*.pdf"))
print("PDF files found:", [f.name for f in pdf_files])

PDF files found: ['Public_251.pdf', 'Public_272.pdf', 'Public_264.pdf', 'Public_260.pdf', 'Public_061.pdf']


In [15]:
from src.services.factories import make_pdf_parser
from src.db.session import get_db_context
from src.models.document import Document

parser = make_pdf_parser()

with get_db_context() as db:
    for pdf_path in pdf_files:
        parsed = parser.parse_pdf(pdf_path)
        doc_id = Path(pdf_path).stem
        # Tạo object Document thử mà chưa commit
        doc = Document(
            doc_id=doc_id,
            filename=parsed["metadata"]["file_name"],
            file_path=pdf_path,
            title=parsed["title"],
            full_text=parsed["full_text"],
            raw_content=parsed["full_text"],
            page_count=parsed["metadata"]["page_count"],
            sections=parsed["sections"],
            tables=parsed["tables"],
            doc_metadata=parsed["metadata"],
            source_folder=Path(pdf_path).parent.name,
            processing_status="completed",
        )
        print(f"Parsed document: {doc_id}, title: {doc.title[:50]}...")

/Users/dohainam/test/rag_mcq_system/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-10-31 00:48:09,149 - INFO - Parsing PDF: data/pdf/Public_251.pdf
2025-10-31 00:48:09,151 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-10-31 00:48:09,195 - INFO - Going to convert document batch...
2025-10-31 00:48:09,195 - INFO - Initializing pipeline for StandardPdfPipeline with options hash 75463f421d05cb4304e1f714cf00d35d
2025-10-31 00:48:09,200 - INFO - Loading plugin 'docling_defaults'
2025-10-31 00:48:09,202 - INFO - Registered picture descriptions: ['vlm', 'api']
2025-10-31 00:48:09,207 - INFO - Loading plugin 'docling_defaults'
2025-10-31 00:48:09,210 - INFO - Registered ocr engines: ['auto', 'easyocr', 'ocrmac', 'rapidocr', 'tesserocr', 'tesseract']
2025-10-31 00:48:09,214 - INFO - Accel

Parsed document: Public_251, title: Public_251...


2025-10-31 00:48:19,141 - INFO - Finished converting document Public_272.pdf in 3.76 sec.
2025-10-31 00:48:19,150 - INFO - Successfully parsed PDF: Public_272, pages: <bound method DoclingDocument.num_pages of DoclingDocument(schema_name='DoclingDocument', version='1.7.0', name='Public_272', origin=DocumentOrigin(mimetype='application/pdf', binary_hash=2132697822789820610, filename='Public_272.pdf', uri=None), furniture=GroupItem(self_ref='#/furniture', parent=None, children=[], content_layer=<ContentLayer.FURNITURE: 'furniture'>, name='_root_', label=<GroupLabel.UNSPECIFIED: 'unspecified'>), body=GroupItem(self_ref='#/body', parent=None, children=[RefItem(cref='#/pictures/0'), RefItem(cref='#/texts/0'), RefItem(cref='#/texts/1'), RefItem(cref='#/texts/2'), RefItem(cref='#/texts/3'), RefItem(cref='#/groups/0'), RefItem(cref='#/texts/7'), RefItem(cref='#/texts/8'), RefItem(cref='#/texts/9'), RefItem(cref='#/texts/10'), RefItem(cref='#/texts/11'), RefItem(cref='#/texts/12'), RefItem(cref

Parsed document: Public_272, title: Public_272...


2025-10-31 00:48:21,283 - INFO - Finished converting document Public_264.pdf in 2.13 sec.
2025-10-31 00:48:21,293 - INFO - Successfully parsed PDF: Public_264, pages: <bound method DoclingDocument.num_pages of DoclingDocument(schema_name='DoclingDocument', version='1.7.0', name='Public_264', origin=DocumentOrigin(mimetype='application/pdf', binary_hash=5119675035389586591, filename='Public_264.pdf', uri=None), furniture=GroupItem(self_ref='#/furniture', parent=None, children=[], content_layer=<ContentLayer.FURNITURE: 'furniture'>, name='_root_', label=<GroupLabel.UNSPECIFIED: 'unspecified'>), body=GroupItem(self_ref='#/body', parent=None, children=[RefItem(cref='#/pictures/0'), RefItem(cref='#/groups/0'), RefItem(cref='#/texts/2'), RefItem(cref='#/texts/3'), RefItem(cref='#/texts/4'), RefItem(cref='#/texts/5'), RefItem(cref='#/texts/6'), RefItem(cref='#/texts/7'), RefItem(cref='#/texts/8'), RefItem(cref='#/texts/9'), RefItem(cref='#/texts/10'), RefItem(cref='#/texts/11'), RefItem(cref=

Parsed document: Public_264, title: Public_264...


2025-10-31 00:48:23,864 - INFO - Finished converting document Public_260.pdf in 2.57 sec.
2025-10-31 00:48:23,875 - INFO - Successfully parsed PDF: Public_260, pages: <bound method DoclingDocument.num_pages of DoclingDocument(schema_name='DoclingDocument', version='1.7.0', name='Public_260', origin=DocumentOrigin(mimetype='application/pdf', binary_hash=11617410907304067118, filename='Public_260.pdf', uri=None), furniture=GroupItem(self_ref='#/furniture', parent=None, children=[], content_layer=<ContentLayer.FURNITURE: 'furniture'>, name='_root_', label=<GroupLabel.UNSPECIFIED: 'unspecified'>), body=GroupItem(self_ref='#/body', parent=None, children=[RefItem(cref='#/pictures/0'), RefItem(cref='#/texts/0'), RefItem(cref='#/texts/1'), RefItem(cref='#/texts/2'), RefItem(cref='#/texts/3'), RefItem(cref='#/texts/4'), RefItem(cref='#/tables/0'), RefItem(cref='#/texts/5'), RefItem(cref='#/groups/0'), RefItem(cref='#/texts/7'), RefItem(cref='#/pictures/1'), RefItem(cref='#/texts/8'), RefItem(cr

Parsed document: Public_260, title: Public_260...


2025-10-31 00:48:24,992 - INFO - Finished converting document Public_061.pdf in 1.12 sec.
2025-10-31 00:48:24,998 - INFO - Successfully parsed PDF: Public_061, pages: <bound method DoclingDocument.num_pages of DoclingDocument(schema_name='DoclingDocument', version='1.7.0', name='Public_061', origin=DocumentOrigin(mimetype='application/pdf', binary_hash=11381279440607448659, filename='Public_061.pdf', uri=None), furniture=GroupItem(self_ref='#/furniture', parent=None, children=[], content_layer=<ContentLayer.FURNITURE: 'furniture'>, name='_root_', label=<GroupLabel.UNSPECIFIED: 'unspecified'>), body=GroupItem(self_ref='#/body', parent=None, children=[RefItem(cref='#/pictures/0'), RefItem(cref='#/texts/0'), RefItem(cref='#/texts/1'), RefItem(cref='#/texts/2'), RefItem(cref='#/groups/0'), RefItem(cref='#/texts/4'), RefItem(cref='#/texts/5'), RefItem(cref='#/texts/6'), RefItem(cref='#/texts/7'), RefItem(cref='#/texts/8'), RefItem(cref='#/texts/9'), RefItem(cref='#/texts/10'), RefItem(cref=

Parsed document: Public_061, title: Public_061...


In [16]:
from pprint import pprint

for pdf_path in pdf_files:
    parsed = parser.parse_pdf(pdf_path)
    pprint(parsed)  # In đẹp toàn bộ dict

2025-10-31 00:48:25,017 - INFO - Parsing PDF: data/pdf/Public_251.pdf
2025-10-31 00:48:25,018 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-10-31 00:48:25,022 - INFO - Going to convert document batch...
2025-10-31 00:48:25,022 - INFO - Processing document Public_251.pdf
2025-10-31 00:48:27,111 - INFO - Finished converting document Public_251.pdf in 2.09 sec.
2025-10-31 00:48:27,117 - INFO - Successfully parsed PDF: Public_251, pages: <bound method DoclingDocument.num_pages of DoclingDocument(schema_name='DoclingDocument', version='1.7.0', name='Public_251', origin=DocumentOrigin(mimetype='application/pdf', binary_hash=828940394524416805, filename='Public_251.pdf', uri=None), furniture=GroupItem(self_ref='#/furniture', parent=None, children=[], content_layer=<ContentLayer.FURNITURE: 'furniture'>, name='_root_', label=<GroupLabel.UNSPECIFIED: 'unspecified'>), body=GroupItem(self_ref='#/body', parent=None, children=[RefItem(cref='#/pictures/0'), RefItem(cref='#/texts/0'), Ref

{'full_text': '<!-- image -->\n'
              '\n'
              '## VIETTEL AI RACE\n'
              '\n'
              '## ACK Flood Attack là gì? Điểm khác biệt gì so với các loại '
              'tấn công DDoS khác?\n'
              '\n'
              'Public 251\n'
              '\n'
              'L ầ n ban hành: 1\n'
              '\n'
              'Các cuộc tấn công DDoS (Distributed Denial of Service) ngày '
              'càng trở nên phổ biến và tinh vi, gây ra nhiều thiệt hại nghiêm '
              'trọng cho hệ thống mạng và dịch vụ trực tuyến. Một trong những '
              'hình thức tấn công DDoS đặc biệt nguy hiểm là tấn công ACK '
              'Flood. Vậy ACK Flood Attack là gì và điểm khác biệt của nó so '
              'với các loại tấn công DDoS khác ra sao?\n'
              '\n'
              '## 1. ACK Flood Attack là gì?\n'
              '\n'
              'ACK Flood Attack là một dạng tấn công mạng thuộc nhóm tấn công '
              'từ chối dịch vụ phân t

2025-10-31 00:48:30,111 - INFO - Finished converting document Public_272.pdf in 2.98 sec.
2025-10-31 00:48:30,117 - INFO - Successfully parsed PDF: Public_272, pages: <bound method DoclingDocument.num_pages of DoclingDocument(schema_name='DoclingDocument', version='1.7.0', name='Public_272', origin=DocumentOrigin(mimetype='application/pdf', binary_hash=2132697822789820610, filename='Public_272.pdf', uri=None), furniture=GroupItem(self_ref='#/furniture', parent=None, children=[], content_layer=<ContentLayer.FURNITURE: 'furniture'>, name='_root_', label=<GroupLabel.UNSPECIFIED: 'unspecified'>), body=GroupItem(self_ref='#/body', parent=None, children=[RefItem(cref='#/pictures/0'), RefItem(cref='#/texts/0'), RefItem(cref='#/texts/1'), RefItem(cref='#/texts/2'), RefItem(cref='#/texts/3'), RefItem(cref='#/groups/0'), RefItem(cref='#/texts/7'), RefItem(cref='#/texts/8'), RefItem(cref='#/texts/9'), RefItem(cref='#/texts/10'), RefItem(cref='#/texts/11'), RefItem(cref='#/texts/12'), RefItem(cref

{'full_text': '<!-- image -->\n'
              '\n'
              '## VIETTEL AI RACE\n'
              '\n'
              '## Các nhóm ki ể m soát\n'
              '\n'
              '## 1. Các nhóm ki ể m soát\n'
              '\n'
              'Trong an toàn thông tin, các biện pháp kiểm soát ( controls ) '
              'được chia thành ba nhóm chính:\n'
              '\n'
              '- Kiểm soát hành chính/Quản lý (Administrative/Managerial '
              'controls)\n'
              '- Kiểm soát kỹ thuật (Technical controls)\n'
              '- Kiểm soát vật lý/Vận hành (Physical/Operational controls)\n'
              '\n'
              '## 1.1 Ki ể m soát hành chính/Qu ả n lý\n'
              '\n'
              'Nhóm này xử lý yếu tố con người trong an toàn thông tin. Bao '
              'gồm chính sách và quy trình quy định cách tổ chức quản lý dữ '
              'liệu, cũng như trách nhiệm của nhân viên trong việc bảo vệ tổ '
              'chức.\n'
              '\n'
     

2025-10-31 00:48:31,840 - INFO - Finished converting document Public_264.pdf in 1.71 sec.
2025-10-31 00:48:31,853 - INFO - Successfully parsed PDF: Public_264, pages: <bound method DoclingDocument.num_pages of DoclingDocument(schema_name='DoclingDocument', version='1.7.0', name='Public_264', origin=DocumentOrigin(mimetype='application/pdf', binary_hash=5119675035389586591, filename='Public_264.pdf', uri=None), furniture=GroupItem(self_ref='#/furniture', parent=None, children=[], content_layer=<ContentLayer.FURNITURE: 'furniture'>, name='_root_', label=<GroupLabel.UNSPECIFIED: 'unspecified'>), body=GroupItem(self_ref='#/body', parent=None, children=[RefItem(cref='#/pictures/0'), RefItem(cref='#/groups/0'), RefItem(cref='#/texts/2'), RefItem(cref='#/texts/3'), RefItem(cref='#/texts/4'), RefItem(cref='#/texts/5'), RefItem(cref='#/texts/6'), RefItem(cref='#/texts/7'), RefItem(cref='#/texts/8'), RefItem(cref='#/texts/9'), RefItem(cref='#/texts/10'), RefItem(cref='#/texts/11'), RefItem(cref=

{'full_text': '<!-- image -->\n'
              '\n'
              'Public 264\n'
              '\n'
              'L ầ n ban hành: 1\n'
              '\n'
              '## VIETTEL AI RACE\n'
              '\n'
              '## LOGIC BOMD LÀ GÌ? CÁCH PHÒNG CH Ố NG LOGIC BOMB HI Ệ U QU Ả\n'
              '\n'
              'Logic bomb là một loại mã độc có thể bị kích hoạt khi những '
              'điều kiện nhất định được thỏa mãn. Đây là một trong những mối '
              'đe dọa lớn đối với an ninh thông tin, đặc biệt khi chúng có khả '
              'năng gây ra thiệt hại nghiêm trọng cho hệ thống máy tính và dữ '
              'liệu.\n'
              '\n'
              '## 1. Logic bomb là gì?\n'
              '\n'
              'Logic bomb là một đoạn mã độc hại được nhúng vào trong một ứng '
              'dụng hoặc chương trình phần mềm. Khi những điều kiện đã được '
              'lập trình sẵn trong mã được đáp ứng, logic bomb sẽ tự động thực '
              'thi và có thể 

2025-10-31 00:48:34,260 - INFO - Finished converting document Public_260.pdf in 2.40 sec.
2025-10-31 00:48:34,269 - INFO - Successfully parsed PDF: Public_260, pages: <bound method DoclingDocument.num_pages of DoclingDocument(schema_name='DoclingDocument', version='1.7.0', name='Public_260', origin=DocumentOrigin(mimetype='application/pdf', binary_hash=11617410907304067118, filename='Public_260.pdf', uri=None), furniture=GroupItem(self_ref='#/furniture', parent=None, children=[], content_layer=<ContentLayer.FURNITURE: 'furniture'>, name='_root_', label=<GroupLabel.UNSPECIFIED: 'unspecified'>), body=GroupItem(self_ref='#/body', parent=None, children=[RefItem(cref='#/pictures/0'), RefItem(cref='#/texts/0'), RefItem(cref='#/texts/1'), RefItem(cref='#/texts/2'), RefItem(cref='#/texts/3'), RefItem(cref='#/texts/4'), RefItem(cref='#/tables/0'), RefItem(cref='#/texts/5'), RefItem(cref='#/groups/0'), RefItem(cref='#/texts/7'), RefItem(cref='#/pictures/1'), RefItem(cref='#/texts/8'), RefItem(cr

{'full_text': '<!-- image -->\n'
              '\n'
              '## VIETTEL AI RACE\n'
              '\n'
              '## Phát tri ển chính sách trong đả m b ả o thông tin\n'
              '\n'
              '## 1. Các y ế u t ố c ủ a chính sách nh ậ n th ức &amp; đào tạ '
              'o\n'
              '\n'
              '## 1.1 T ổ ng quan\n'
              '\n'
              'Đối với mỗi rủi ro và mối đe dọa đã xác định trong Miền Người '
              'Dùng và Miền Máy Trạm, hãy xác định một kiểm soát bảo mật hoặc '
              'biện pháp đối phó bảo mật có thể giúp giảm thiểu rủi ro hoặc '
              'mối đe dọa.\n'
              '\n'
              '| Domain             | Rủi ro &Mối đe '
              'dọa                                                                                                 '
              '| Chiến lược/Tài liệu Giảm thiểu Rủi ro   |\n'
              '|--------------------|----------------------------------------------------------------------

2025-10-31 00:48:35,229 - INFO - Finished converting document Public_061.pdf in 0.95 sec.
2025-10-31 00:48:35,235 - INFO - Successfully parsed PDF: Public_061, pages: <bound method DoclingDocument.num_pages of DoclingDocument(schema_name='DoclingDocument', version='1.7.0', name='Public_061', origin=DocumentOrigin(mimetype='application/pdf', binary_hash=11381279440607448659, filename='Public_061.pdf', uri=None), furniture=GroupItem(self_ref='#/furniture', parent=None, children=[], content_layer=<ContentLayer.FURNITURE: 'furniture'>, name='_root_', label=<GroupLabel.UNSPECIFIED: 'unspecified'>), body=GroupItem(self_ref='#/body', parent=None, children=[RefItem(cref='#/pictures/0'), RefItem(cref='#/texts/0'), RefItem(cref='#/texts/1'), RefItem(cref='#/texts/2'), RefItem(cref='#/groups/0'), RefItem(cref='#/texts/4'), RefItem(cref='#/texts/5'), RefItem(cref='#/texts/6'), RefItem(cref='#/texts/7'), RefItem(cref='#/texts/8'), RefItem(cref='#/texts/9'), RefItem(cref='#/texts/10'), RefItem(cref=

{'full_text': '<!-- image -->\n'
              '\n'
              '## VIETTEL AI RACE\n'
              '\n'
              '## KHẢO SÁT ĐẶC TẢ VÀ MÃ NGUỒN\n'
              '\n'
              'TD061\n'
              '\n'
              'Lần ban hành: 1\n'
              '\n'
              'Chương này giới thiệu các kỹ thuật khảo sát đặc tả và mã nguồn. '
              'Mã nguồn được phát triển dựa trên đặc tả và vì thế việc khảo '
              'sát đặc tả cần được tiến hành trước khi phát triển mã nguồn để '
              'tránh các rủi ro về các lỗi có thể có trong đặc tả về sản phẩm '
              'phần mềm. Vì đặc tả không thể thực thi được trên máy nên chúng '
              'ta chỉ có thể phát hiện các lỗi bằng các kỹ thuật khảo sát đặc '
              'tả. Tuy nhiên, mã nguồn thì có thể thực thi được và có thể được '
              'kiểm thử thông qua việc chạy trên máy. Vì vậy, liệu chúng ta có '
              'cần phải khảo sát mã nguồn trước không? Câu trả lời là rất cần '
       

In [17]:
from src.services.factories import make_text_chunker

chunker = make_text_chunker()
for doc in db.query(Document).all():
    doc_data = {
        "title": doc.title,
        "full_text": doc.full_text,
        "sections": doc.sections or [],
    }
    chunks = chunker.chunk_document(doc_data, doc.doc_id)
    print(f"Document {doc.doc_id} -> {len(chunks)} chunks")


2025-10-31 00:48:35,775 INFO sqlalchemy.engine.Engine select pg_catalog.version()


2025-10-31 00:48:35,775 - INFO - select pg_catalog.version()


2025-10-31 00:48:35,776 INFO sqlalchemy.engine.Engine [raw sql] {}


2025-10-31 00:48:35,776 - INFO - [raw sql] {}


2025-10-31 00:48:35,778 INFO sqlalchemy.engine.Engine select current_schema()


...

2025-10-31 00:48:35,788 INFO sqlalchemy.engine.Engine SELECT documents.id AS documents_id, documents.doc_id AS documents_doc_id, documents.filename AS documents_filename, documents.file_path AS documents_file_path, documents.title AS documents_title, documents.full_text AS documents_full_text, documents.raw_content AS documents_raw_content, documents.page_count AS documents_page_count, documents.file_size AS documents_file_size, documents.sections AS documents_sections, documents.tables AS documents_tables, documents.doc_metadata AS documents_doc_metadata, documents.source_folder AS documents_source_folder, documents.processing_status AS documents_processing_status, documents.error_message AS documents_error_message, documents.created_at AS documents_created_at, documents.updated_at AS documents_updated_at, documents.processed_at AS documents_processed_at 
FROM documents


2025-10-31 00:48:35,788 - INFO - SELECT documents.id AS documents_id, documents.doc_id AS documents_doc_id, documents.filename AS documents_filename, documents.file_path AS documents_file_path, documents.title AS documents_title, documents.full_text AS documents_full_text, documents.raw_content AS documents_raw_content, documents.page_count AS documents_page_count, documents.file_size AS documents_file_size, documents.sections AS documents_sections, documents.tables AS documents_tables, documents.doc_metadata AS documents_doc_metadata, documents.source_folder AS documents_source_folder, documents.processing_status AS documents_processing_status, documents.error_message AS documents_error_message, documents.created_at AS documents_created_at, documents.updated_at AS documents_updated_at, documents.processed_at AS documents_processed_at 
FROM documents


2025-10-31 00:48:35,789 INFO sqlalchemy.engine.Engine [generated in 0.00083s] {}


2025-10-31 00:48:35,789 - INFO - [generated in 0.00083s] {}


ProgrammingError: (psycopg2.errors.UndefinedTable) relation "documents" does not exist
LINE 2: FROM documents
             ^

[SQL: SELECT documents.id AS documents_id, documents.doc_id AS documents_doc_id, documents.filename AS documents_filename, documents.file_path AS documents_file_path, documents.title AS documents_title, documents.full_text AS documents_full_text, documents.raw_content AS documents_raw_content, documents.page_count AS documents_page_count, documents.file_size AS documents_file_size, documents.sections AS documents_sections, documents.tables AS documents_tables, documents.doc_metadata AS documents_doc_metadata, documents.source_folder AS documents_source_folder, documents.processing_status AS documents_processing_status, documents.error_message AS documents_error_message, documents.created_at AS documents_created_at, documents.updated_at AS documents_updated_at, documents.processed_at AS documents_processed_at 
FROM documents]
(Background on this error at: https:/

In [ ]:
from src.services.factories import make_embeddings_service
import asyncio

emb_service = make_embeddings_service()
texts = [chunk["chunk_text"] for chunk in chunks]
embeddings = asyncio.run(emb_service.embed_texts(texts))
print(f"Generated {len(embeddings)} embeddings, first vector len = {len(embeddings[0])}")


In [ ]:
from src.services.factories import make_opensearch_client
import asyncio

opensearch = make_opensearch_client()
asyncio.run(opensearch.create_index(dimensions=settings.embeddings.dimensions))

# Index thử 1 chunk
doc_to_index = {
    "chunk_id": chunks[0]["chunk_id"],
    "document_id": "test_doc",
    "chunk_text": chunks[0]["chunk_text"],
    "embedding": embeddings[0].tolist(),
}
result = asyncio.run(opensearch.index_chunks_bulk([doc_to_index]))
print("Index result:", result)
